In [26]:
import yfinance as yf
import pandas as pd
import numpy as np
import os
import pickle
import joblib
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

class CandleFit:
    def __init__(self, ticker: str, period: str = '5y'):
        """
        Initializes the CandleFit object with a specified ticker and period.

        Parameters:
        ticker (str): The ticker symbol of the stock.
        period (str): The period for which to download historical data.
        """
        self.ticker = ticker
        self.period = period
        self.data = self.get_ticker()
        self.features = self.get_price_features()
        self.threshold_dict: dict = None
      

    def get_ticker(self):
        """
        Downloads historical data for the specified ticker and period.

        Returns:
        pd.DataFrame: A DataFrame containing the historical data with formatted dates and column names.
        """
        try:
            ticker_obj = yf.Ticker(self.ticker)
            hist = ticker_obj.history(period=self.period)
            hist.index = pd.to_datetime(hist.index).strftime('%Y-%m-%d')
            hist.columns = [col.lower() for col in hist.columns]
            return hist
        except Exception as e:
            print(f"Error downloading aux: {e}")
            return pd.DataFrame()
    
    @staticmethod
    def load_dict(key: str) -> dict:
        """
        Loads a dictionary from a pickle file based on the provided key.

        Parameters:
        key (str): The key to search for in the dictionary.

        Returns:
        dict: The dictionary associated with the provided key.

        Raises:
        FileNotFoundError: If the pickle file is not found.
        KeyError: If the key is not found in the dictionary.
        ValueError: If there is an error loading the pickle file.
        """
        filepath = os.path.join('..', 'pkl', 'threshold_dicts.pkl')
        try:
            with open(filepath, 'rb') as file:
                data = pickle.load(file)
            
            for item in data:
                if key in item:
                    return item[key]
            raise KeyError(f"Key '{key}' not found in the threshold dictionary.")
        except FileNotFoundError:
            raise FileNotFoundError(f"The file '{filepath}' was not found.")
        except pickle.PickleError:
            raise ValueError("Error occurred while loading the pickle file.")
        
    def get_price_features(self):
        """
        Calculates various price and volume features from historical data.

        Returns:
        pd.DataFrame: A DataFrame containing the calculated features.
        """
        aux = self.data.copy()
        df = pd.DataFrame(index=aux.index)
        df.index = pd.to_datetime(df.index).strftime('%Y-%m-%d')
        df['return_rate'] = aux['close'].pct_change()
        df['volume_change'] = aux['volume'].diff()
        df['volume_var'] = aux['volume'].pct_change() + 1
        df['price_range'] = aux['high'] - aux['low']
        df['price_var'] = df['price_range'] / aux['low']
        df['price_change'] = aux['close'] - aux['open']
        df['close_vol'] = aux['close'].expanding().std()
        df['low_vol'] = aux['low'].expanding().std()
        df['high_vol'] = aux['high'].expanding().std()
        df['open_vol'] = aux['open'].expanding().std()
        df['upper_wick'] = aux['high'] - aux[['open', 'close']].max(axis=1)
        df['lower_wick'] = aux[['open', 'close']].min(axis=1) - aux['low']
        df['wick_change'] = df['upper_wick'] - df['lower_wick']
        df['wick_var'] = df['wick_change'] / df['lower_wick']
        df['std_wick'] = df['wick_change'].abs().expanding().std()
        df = df.apply(pd.to_numeric, errors='coerce')
        df = pd.concat([aux, df], axis=1)
        
        return df

    def get_candle_features(self, 
                            doji_threshold: float =  0.002, 
                            bullish_threshold : float =  0.005,
                            bearish_threshold: float =  0.005,
                            volatility_window: int = 7):
                            
 
        """
        Calculates various candlestick features based on price data and a threshold dictionary.

        Parameters:
        threshold_dict (dict): A dictionary containing thresholds for calculating features. If None, it loads the default dictionary.

        Returns:
        pd.DataFrame: A DataFrame containing the calculated candlestick features.
        """
        
        aux = self.features.copy()
        aux = aux.loc[:, ~aux.columns.duplicated()]
        df = pd.DataFrame(index=aux.index)
        df[f'std_volatility_window'] = aux['price_change'].rolling(window=volatility_window).std().abs()
        df['bearish_threshold'] = pd.to_numeric(bearish_threshold * df[f'std_volatility_window'], errors='coerce').fillna(0)
        df['bullish_threshold'] = pd.to_numeric(bullish_threshold * df[f'std_volatility_window'], errors='coerce').fillna(0)
        df['is_bearish'] = (aux['close'] <= (aux['open'] - df['bearish_threshold'])).astype(int)
        df['is_bullish'] = (aux['close'] >= (aux['open'] + df['bullish_threshold'])).astype(int)
        df['is_doji'] = (abs(aux['close'] - aux['open']) <= doji_threshold).astype(int)
        df['is_bearish_open_gap'] = (aux['open'] < aux['close'].shift(1)).astype(int)
        df['is_bullish_open_gap'] = (aux['open'] > aux['close'].shift(1)).astype(int)
    
        df = pd.concat([aux, df], axis=1)
        self.features = df
        return  self.features

    def fit_morning_star(self,
                         doji_threshold: float = 0.002,
                         bullish_to_bearish_ratio : float = 0.33,
                         bearish_threshold: float = 0.2):
                  
        """
        Identifies the morning star candlestick pattern in the historical data.

        Parameters:
        doji_threshold (float): The threshold for the price change to identify a doji candle. Default is 0.002.
        bullish_threshold (float): The ratio of the price change on the third day to the price change on the first day to identify a bullish candle. Default is 0.33.
        bearish_threshold (float): The threshold to identify a bearish candle on the first day. Default is 0.2.
        volatility_window (int): The window size (in days) for calculating the volatility of the historical data. Default is 7.

        Returns:
        pd.DataFrame: A DataFrame with a column indicating the presence of the morning star pattern.
        """
        df = self.get_candle_features()
        df = df.loc[:, ~df.columns.duplicated(keep='last')]

        # Condition 1: Two days ago was a bearish candle and the close of that day is lower than the open of that day adjusted by the threshold
        df['is_bearish_morning_star'] = ((df['is_bearish'].shift(2) == 1) & 
                                        (df['close'].shift(2) + bearish_threshold * df['price_change'].shift(2) 
                                        <= df['open'].shift(2))).astype(int)

        # Condition 2: The previous day was a doji candle and had a bearish open gap
        df['is_bearish_open_gap_morning_star'] = (df['is_bearish_open_gap'].shift(1) == 1).astype(int)
        df['is_doji_morning_star'] = (df['price_change'].shift(1).abs() <= doji_threshold ).astype(int)

        # Condition 3: Today is a bullish candle and the price change from two days ago to today is significant
        df['is_bullish_morning_star'] = ((df['is_bullish'] == 1) & \
                                        (df['close'] - df['close'].shift(2) >= bullish_to_bearish_ratio)
                                        * df['price_change'].shift(2).abs()).astype(int)

        # Combine all conditions to determine the morning star pattern
        df['is_morning_star'] = df[['is_bearish_morning_star', 'is_bearish_open_gap_morning_star', 'is_doji_morning_star', 
                                    'is_bullish_morning_star']].all(axis=1).astype(int)

        self.features = df
        return self.features
    
    
    def fit_hammer(self,
                   lower_wick_to_price_change_ratio: float = 3,
                   upper_wick_to_price_change_ratio: float = 0.02,
                   volatility_window: int = None):
        """
        Identifies the hammer candlestick pattern in the historical data.

        Parameters:
        lower_wick_to_price_change_ratio (float): The minimum ratio of the lower wick length to the price change to identify a hammer handle. Default is 4.
        upper_wick_to_price_change_ratio (float): The maximum ratio of the upper wick length to the price change to identify a hammer head. Default is 0.05.

        Returns:
        pd.DataFrame: A DataFrame with columns indicating the presence of the hammer pattern and its components.
        """
        df = self.get_candle_features(volatility_window).copy()
        df = df.loc[:, ~df.columns.duplicated(keep='last')]

        # Condition 1: Two days ago was a bearish candle and the close of that day is lower than the open of that day adjusted by the threshold
        df['is_hammer_head'] =  (df['upper_wick'] <= upper_wick_to_price_change_ratio * df['std_volatility_window']).astype(int)
        df['is_hammer_handle'] =  (df['lower_wick'] / df['price_change'].abs() >= df['std_volatility_window'] * lower_wick_to_price_change_ratio).astype(int)
        df['is_hammer'] = df[['is_hammer_head', 'is_hammer_handle']].all(axis=1).astype(int)

        self.features = df
        return self.features

    
    def get_movings(self, short:int = None, long:int = None, strategy: str = 'test'):

        """
        Calculates buy and sell signals based on moving averages for different strategies and parameters.

        Parameters:
        - threshold_dict (dict, optional): Dictionary containing moving average settings for different strategies. If any of parameters is None,
        it will be loaded with `self.load_dict(key='rolling_cross_dict')` and available strategies will be measured.  
        - short (int, optional): Time window for the short moving average. Not used directly in the function.
        - long_ (int, optional): Time window for the long moving average. Not used directly in the function.
        - signal_window (int, optional): Time window for signal calculation. Not used directly in the function.

        Returns:
        - pd.DataFrame: DataFrame with additional columns for moving averages and buy/sell signals.
        """
        df = self.get_candle_features()  
        df = df.loc[:, ~df.columns.duplicated(keep='last')]
        
        if (short is None) != (long is None):
            raise ValueError("Please set both short and long to valid int or set both to None.")
        elif all(x is not None for x in (short, long)):
            threshold_dict = {f'{strategy}_{short}_{long}': {'short': short, 'long': long}}
        else:
            display('Loading standard strategies')
            threshold_dict = self.load_dict(key='rolling_cross_dict')

        tolerance = 0.015 * df['close_vol']  
        for key, value in threshold_dict.items():
            strategy = key.split('_')[0] if '_' in key else key
            short = value['short']
            long = value['long']
            
            df[f'{strategy}_short_{short}'] = df['close'].rolling(window=short, min_periods=1).mean()
            df[f'{strategy}_long_{long}'] = df['close'].rolling(window=long, min_periods=1).mean()
            
            # Detectar cruzamento de médias móveis com margem de tolerância
            buy = (df[f'{strategy}_short_{short}'] > df[f'{strategy}_long_{long}'] + tolerance) & \
                (df[f'{strategy}_short_{short}'].shift(1) < df[f'{strategy}_long_{long}'].shift(1) + tolerance)
            
            sell = (df[f'{strategy}_short_{short}'] < df[f'{strategy}_long_{long}'] - tolerance) & \
                (df[f'{strategy}_short_{short}'].shift(1) > df[f'{strategy}_long_{long}'].shift(1) - tolerance)
            
            df[f'{strategy}_{short}_{long}'] = np.where(buy, 1, np.where(sell, -1, 0))
                        

        self.features = df
        self.threshold_dict = threshold_dict  
        return self.features          

    def candlestick_chart(self, 
                      key: str = 'is_morning_star', 
                      plot_type: str = 'pattern',
                      height: int = 1200, 
                      offset: float = 12):
        """
        Generates a candlestick chart with optional pattern or price action markers.

        Parameters:
        - key (str): Key for identifying the pattern or price action indicators. Defaults to 'is_morning_star'.
        - plot_type (str): Type of plot to generate. Options are 'pattern' or 'price_action'. Defaults to 'pattern'.
        - height (int): Height of the plot in pixels. Defaults to 900.
        - offset (float): Vertical offset for pattern markers. Defaults to 12.

        Returns:
        - go.Figure: A Plotly Figure object with the candlestick chart.
        """
        
        aux = self.features.copy()
        aux = aux.loc[:, ~aux.columns.duplicated(keep='last')]
        
        if key not in aux.columns:
            print(f"Key '{key}' not found in aux columns.")
            return None

        trace = go.Candlestick(
            x=aux.index,
            open=aux["open"],
            high=aux["high"],
            low=aux["low"],
            close=aux["close"],
            name=self.ticker,
            yaxis="y"
        )
        
        volume_colors = ['green' if aux['volume'][i] > aux['volume'][i-1] else 'red' for i in range(1, len(aux))]
        volume_colors.insert(0, 'green')
        volume_trace = go.Bar(
                x=aux.index,
                y=aux['volume'],
                marker_color=volume_colors,
                name='Volume',
                yaxis="y2"
            )
            
        
        if plot_type == 'pattern':     
            markers = aux[aux[key] == 1].copy()
            markers['close'] = markers['low'] - offset
            markers = markers.dropna(subset=['close'])
            marker_trace = go.Scatter(
                x=markers.index,
                y=markers['close'],
                mode='markers',
                marker=dict(
                    color='blue',
                    size=8,
                    symbol='triangle-up'
                ),
                name=key,
                yaxis="y")
            
            data = [trace, marker_trace, volume_trace]
            
        elif plot_type == 'moving_cross':
            strategy, short, long = key.split('_')
            short_col = f'{strategy}_short_{short}'
            long_col = f'{strategy}_long_{long}'
            signal_col = f'{strategy}_{short}_{long}'
            short_trace = go.Scatter(
                x=aux.index,
                y=aux[short_col],
                mode='lines',
                name=f'{strategy.capitalize()} Short {short}',
                line=dict(color='purple')  
            )
            long_trace = go.Scatter(
                x=aux.index,
                y=aux[long_col],
                mode='lines',
                name=f'{strategy.capitalize()} Long {long}',
                line=dict(color='orange')  
            )

            markers = aux[aux[signal_col] != 0].copy()  
            markers['close'] = markers.apply(
                lambda row: row['low'] - offset if row[signal_col] == 1 else row['high'] + offset,
                axis=1
            )
            markers['color'] = markers[signal_col].apply(lambda x: 'green' if x == 1 else 'red')
            markers['symbol'] = markers[signal_col].apply(lambda x: 'arrow-up' if x == 1 else 'arrow-down')
            
            marker_trace = go.Scatter(
                x=markers.index,
                y=markers['close'],
                mode='markers',
                marker=dict(
                    color=markers['color'],
                    size=8,
                    symbol=markers['symbol']
                ),
                name='Signal',
                yaxis="y"
            )
            
           
            data = [trace, short_trace, long_trace, marker_trace, volume_trace]
        
        layout = go.Layout(
            title=f"{self.ticker} Candlestick Chart markers: {key.capitalize()}",
            xaxis=dict(title="Date"),
            yaxis=dict(title="Price", domain=[0.3, 1]),
            yaxis2=dict(title="Volume", domain=[0, 0.2]),
            height=height,
            barmode='relative'
        )
        
        fig = go.Figure(data=data, layout=layout)
        return fig
        
    @staticmethod      
    def get_trades(df, 
               strategy: str = None,
               reward_risk_ratio: list = None, 
               price_col: str = 'close',
               trade_period=7,
               target_return=0.01):
    
        cols = ['open', 'high', 'low', 'close']
        shifted_cols = []
        stop_loss_cols = []

        for col in cols:
            for i in range(2, trade_period + 1):
                df[f'{col}_{i}'] = df[col].shift(-i)
                shifted_cols.append(f'{col}_{i}')
        
        has_signals = (df[strategy] == 1) | (df[strategy] == -1)
        if not has_signals.any():
            return "The strategy did not generate buy or sell signals."
        
        df = df[cols + [strategy] + shifted_cols][has_signals].copy()
        df['side'] = df[strategy].apply(lambda x: 'long' if x == 1 else 'short')
        df['target_price'] = df.apply(lambda row: (1 + target_return) * row[price_col] if row[strategy] == 1 else (1 - target_return) * row[price_col], axis=1)
        if reward_risk_ratio is None:      
            reward_risk_ratio = np.arange(0.25, 2.25, 0.25)
        
        for rw in reward_risk_ratio:
            df[f'stop_loss_reward_risk_ratio_{rw}'] = np.where(df['side'] == 'long', 
                                                                df[price_col] * (1 - target_return * rw),
                                                                df[price_col] * (1 + target_return * rw))
            stop_loss_cols.append(f'stop_loss_reward_risk_ratio_{rw}')

        
        for loss_col in stop_loss_cols:
            df[f'out_of_position_{loss_col}'] = np.nan
            df[f'result_{loss_col}'] = np.nan
            df[f'result_{loss_col}_value'] = np.nan
            
            for index, row in df.iterrows():
                stop_loss_val = row[loss_col]
                target_price = row['target_price']
                entry_price = row[price_col] 
                
                for i in range(2, trade_period + 1):
                    next_open = row[f'open_{i}']
                    next_high = row[f'high_{i}']
                    next_low = row[f'low_{i}']
                    next_close = row[f'close_{i}']
                    
                    if row['side'] == 'long':
                        price_vars = [next_open, next_low, next_high, next_close]
                        for price_var in price_vars:
                            if price_var > target_price:
                                df.at[index, f'out_of_position_{loss_col}'] = i
                                df.at[index, f'result_{loss_col}'] = 'profit'
                                df.at[index, f'result_{loss_col}_value'] = price_var - entry_price
                                break
                            elif price_var < stop_loss_val:
                                df.at[index, f'out_of_position_{loss_col}'] = i
                                df.at[index, f'result_{loss_col}'] = 'loss'
                                df.at[index, f'result_{loss_col}_value'] = entry_price - price_var
                                break
                    else:  # 'short'
                        price_vars = [next_open, next_high, next_low, next_close]
                        for price_var in price_vars:
                            if price_var < target_price:
                                df.at[index, f'out_of_position_{loss_col}'] = i
                                df.at[index, f'result_{loss_col}'] = 'profit'
                                df.at[index, f'result_{loss_col}_value'] = entry_price - price_var
                                break
                            elif price_var > stop_loss_val:
                                df.at[index, f'out_of_position_{loss_col}'] = i
                                df.at[index, f'result_{loss_col}'] = 'loss'
                                df.at[index, f'result_{loss_col}_value'] = entry_price - price_var
                                break
                    
                    if not np.isnan(df.at[index, f'out_of_position_{loss_col}']):
                        break
                
        return df

In [27]:

ticker = CandleFit('GOOG')
ticker.get_candle_features()

,open,high,low,close,volume,dividends,stock splits,return_rate,volume_change,volume_var,price_range,price_var,price_change,close_vol,low_vol,high_vol,open_vol,upper_wick,lower_wick,wick_change,wick_var,std_wick,std_volatility_window,bearish_threshold,bullish_threshold,is_bearish,is_bullish,is_doji,is_bearish_open_gap,is_bullish_open_gap
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-08-07,57.734299,58.855273,57.415863,58.632778,28886000,0.0,0.0,NaN,NaN,NaN,1.439410,0.025070,0.898479,NaN,NaN,NaN,NaN,0.222495,0.318436,-0.095941,-0.301289,NaN,NaN,0.000000,0.000000,0,1,0,0,0
2019-08-08,59.074273,60.182014,58.584333,60.171528,29360000,0.0,0.0,0.026244,474000.0,1.016409,1.597681,0.027271,1.097254,1.088060,0.826233,0.938148,0.947505,0.010486,0.489940,-0.479454,-0.978597,0.271185,NaN,0.000000,0.000000,0,1,0,0,1
2019-08-09,59.831418,60.125582,59.112883,59.332985,21314000,0.0,0.0,-0.013936,-8046000.0,0.725954,1.012699,0.017132,-0.498433,0.770411,0.868386,0.750235,1.061972,0.294164,0.220102,0.074062,0.336490,0.228000,NaN,0.000000,0.000000,1,0,0,1,0
2019-08-12,58.893482,59.180658,58.317237,58.668739,20060000,0.0,0.0,-0.011195,-1254000.0,0.941165,0.863420,0.014806,-0.224743,0.722385,0.709544,0.669489,0.867123,0.287176,0.351502,-0.064327,-0.183005,0.201107,NaN,0.000000,0.000000,1,0,0,1,0
2019-08-13,58.506426,60.170529,58.506426,59.795456,25888000,0.0,0.0,0.019205,5828000.0,1.290528,1.664103,0.028443,1.289030,0.679658,0.618078,0.636022,0.769639,0.375073,0.000000,0.375073,inf,0.195104,NaN,0.000000,0.000000,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-31,174.919998,175.910004,171.720001,173.149994,15650200,0.0,0.0,0.007506,1968800.0,1.143903,4.190002,0.024400,-1.770004,32.206053,31.973704,32.448253,32.224491,0.990005,1.429993,-0.439987,-0.307685,0.554908,1.932256,0.009661,0.009661,1,0,0,0,1
2024-08-01,171.979996,175.679993,170.509995,172.449997,17177800,0.0,0.0,-0.004043,1527600.0,1.097609,5.169998,0.030321,0.470001,32.240163,32.007160,32.485072,32.258006,3.229996,1.470001,1.759995,1.197274,0.555682,2.001021,0.010005,0.010005,0,1,0,1,0
2024-08-02,168.190002,170.210007,166.389999,168.399994,18907800,0.0,0.0,-0.023485,1730000.0,1.100711,3.820007,0.022958,0.209991,32.268159,32.034434,32.513556,32.285817,1.810013,1.800003,0.010010,0.005561,0.555696,2.050034,0.010250,0.010250,0,1,0,1,0


In [28]:
ticker.fit_hammer()



,open,high,low,close,volume,dividends,stock splits,return_rate,volume_change,volume_var,price_range,price_var,price_change,close_vol,low_vol,high_vol,open_vol,upper_wick,lower_wick,wick_change,wick_var,std_wick,std_volatility_window,bearish_threshold,bullish_threshold,is_bearish,is_bullish,is_doji,is_bearish_open_gap,is_bullish_open_gap,is_hammer_head,is_hammer_handle,is_hammer
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-08-07,57.734299,58.855273,57.415863,58.632778,28886000,0.0,0.0,NaN,NaN,NaN,1.439410,0.025070,0.898479,NaN,NaN,NaN,NaN,0.222495,0.318436,-0.095941,-0.301289,NaN,NaN,0.000000,0.000000,0,1,0,0,0,0,0,0
2019-08-08,59.074273,60.182014,58.584333,60.171528,29360000,0.0,0.0,0.026244,474000.0,1.016409,1.597681,0.027271,1.097254,1.088060,0.826233,0.938148,0.947505,0.010486,0.489940,-0.479454,-0.978597,0.271185,NaN,0.000000,0.000000,0,1,0,0,1,0,0,0
2019-08-09,59.831418,60.125582,59.112883,59.332985,21314000,0.0,0.0,-0.013936,-8046000.0,0.725954,1.012699,0.017132,-0.498433,0.770411,0.868386,0.750235,1.061972,0.294164,0.220102,0.074062,0.336490,0.228000,NaN,0.000000,0.000000,1,0,0,1,0,0,0,0
2019-08-12,58.893482,59.180658,58.317237,58.668739,20060000,0.0,0.0,-0.011195,-1254000.0,0.941165,0.863420,0.014806,-0.224743,0.722385,0.709544,0.669489,0.867123,0.287176,0.351502,-0.064327,-0.183005,0.201107,NaN,0.000000,0.000000,1,0,0,1,0,0,0,0
2019-08-13,58.506426,60.170529,58.506426,59.795456,25888000,0.0,0.0,0.019205,5828000.0,1.290528,1.664103,0.028443,1.289030,0.679658,0.618078,0.636022,0.769639,0.375073,0.000000,0.375073,inf,0.195104,NaN,0.000000,0.000000,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-31,174.919998,175.910004,171.720001,173.149994,15650200,0.0,0.0,0.007506,1968800.0,1.143903,4.190002,0.024400,-1.770004,32.206053,31.973704,32.448253,32.224491,0.990005,1.429993,-0.439987,-0.307685,0.554908,1.932256,0.009661,0.009661,1,0,0,0,1,0,0,0
2024-08-01,171.979996,175.679993,170.509995,172.449997,17177800,0.0,0.0,-0.004043,1527600.0,1.097609,5.169998,0.030321,0.470001,32.240163,32.007160,32.485072,32.258006,3.229996,1.470001,1.759995,1.197274,0.555682,2.001021,0.010005,0.010005,0,1,0,1,0,0,0,0
2024-08-02,168.190002,170.210007,166.389999,168.399994,18907800,0.0,0.0,-0.023485,1730000.0,1.100711,3.820007,0.022958,0.209991,32.268159,32.034434,32.513556,32.285817,1.810013,1.800003,0.010010,0.005561,0.555696,2.050034,0.010250,0.010250,0,1,0,1,0,0,1,0


In [14]:
df =ticker.features.copy()

In [15]:
ticker.candlestick_chart(key='is_hammer', plot_type='pattern')

In [16]:
ticker.features.is_hammer.value_counts()

is_hammer
0    1257
1       1
Name: count, dtype: int64

In [17]:
def get_trades(df, 
               strategy: str = None,
               reward_risk_ratio: list = None, 
               price_col: str = 'close',
               trade_period=7,
               target_return=0.01):
    
    cols = ['open', 'high', 'low', 'close']
    shifted_cols = []
    stop_loss_cols = []
    
    # Criar colunas deslocadas
    for col in cols:
        for i in range(2, trade_period + 1):
            df[f'{col}_{i}'] = df[col].shift(-i)
            shifted_cols.append(f'{col}_{i}')
    
    # Verificar se há sinais
    has_signals = (df[strategy] == 1) | (df[strategy] == -1)
    if not has_signals.any():
        return "The strategy did not generate buy or sell signals."
    
    df = df[cols + [strategy] + shifted_cols][has_signals].copy()
    df['side'] = df[strategy].apply(lambda x: 'long' if x == 1 else 'short')
    df['target_price'] = df.apply(lambda row: (1 + target_return) * row[price_col] if row[strategy] == 1 else (1 - target_return) * row[price_col], axis=1)
    if reward_risk_ratio is None:      
        reward_risk_ratio = np.arange(0.25, 2.25, 0.25)
    
    for rw in reward_risk_ratio:
        df[f'stop_loss_reward_risk_ratio_{rw}'] = np.where(df['side'] == 'long', 
                                                            df[price_col] * (1 - target_return * rw),
                                                            df[price_col] * (1 + target_return * rw))
        stop_loss_cols.append(f'stop_loss_reward_risk_ratio_{rw}')

    
    for loss_col in stop_loss_cols:
        df[f'out_of_position_{loss_col}'] = np.nan
        df[f'result_{loss_col}'] = np.nan
        df[f'result_{loss_col}_value'] = np.nan
        
        for index, row in df.iterrows():
            stop_loss_val = row[loss_col]
            target_price = row['target_price']
            entry_price = row[price_col] 
            
            for i in range(2, trade_period + 1):
                next_open = row[f'open_{i}']
                next_high = row[f'high_{i}']
                next_low = row[f'low_{i}']
                next_close = row[f'close_{i}']
                
                if row['side'] == 'long':
                    price_vars = [next_open, next_low, next_high, next_close]
                    for price_var in price_vars:
                        if price_var > target_price:
                            df.at[index, f'out_of_position_{loss_col}'] = int(i)
                            df.at[index, f'result_{loss_col}'] = 'profit'
                            df.at[index, f'result_{loss_col}_value'] = price_var - entry_price
                            break
                        elif price_var < stop_loss_val:
                            df.at[index, f'out_of_position_{loss_col}'] = int(i)
                            df.at[index, f'result_{loss_col}'] = 'loss'
                            df.at[index, f'result_{loss_col}_value'] = price_var - entry_price
                            break
                else:  # 'short'
                    price_vars = [next_open, next_high, next_low, next_close]
                    for price_var in price_vars:
                        if price_var < target_price:
                            df.at[index, f'out_of_position_{loss_col}'] =int(i)
                            df.at[index, f'result_{loss_col}'] = 'profit'
                            df.at[index, f'result_{loss_col}_value'] = entry_price - price_var
                            break
                        elif price_var > stop_loss_val:
                            df.at[index, f'out_of_position_{loss_col}'] = int(i)
                            df.at[index, f'result_{loss_col}'] = 'loss'
                            df.at[index, f'result_{loss_col}_value'] = entry_price - price_var
                            break
                
                if not np.isnan(df.at[index, f'out_of_position_{loss_col}']):
                    break
            
    return df
